In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("dataset/DATA_copy_columndevided.csv")

In [3]:
X = dataset.iloc[:, 1:32].values
y = dataset.iloc[:, 32].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train,y_train)